[1] Dynamics of offshore structures, J. F. Wilson (Chapter 10)

In [1]:
import numpy as np
from random import  uniform
from sympy import Sum, cos, pi, diff, Symbol,lambdify, solve
from sympy.abc import a
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
A = 0.78
B = 0.0311
B0 = 122
c = 5.55
CA = 1
CM = 2
d = 1067
D = 9.144
e1 = 0.093
e2 = 0.05
E = 2.07e11
EI = 3.94e11
g = 9.81
K = 1.13e8
L = 61 # to 1037 m
M0 = 4.54e4 # or 2.27e5
M1 = 0.6*M0
m_hat = 6.43e4
m1_hat = 1.25e3
N = 15
rho = 962
omega1 = 0.1
omega_b = 0.503
omega_N = 1.5
omega_p = 0.4

In [3]:
# Pierson-Moskowitz spectrum
def S_eta(omega):
    return A*omega**(-5)*np.exp(-B/omega**4)

In [4]:
def eta(omega_n, del_omega):
    return (2*S_eta(omega_n)*del_omega)**0.5

In [5]:
eta(omega_N, omega_N/N)

0.14288945193592165

In [6]:
def R(omega_n):
    if 0<omega_n<=omega_b :
        return 1
    elif omega_b<omega_n<=omega_N:
        return 1-(omega_n-omega_b)/(omega_N-omega_b)
    else:
        return 0

In [7]:
# heave motion of the barge
def H(t, del_omega):
    return np.sum(R(i*del_omega)*eta(i*del_omega,del_omega)*cos(i*del_omega*t+uniform(0, 2*pi)) for i in range(1,N+1) )

In [8]:
def P(x,t):
    x_=Symbol('x_')
    t_=Symbol('t_')
    def P_(x_,t_):
        return M1*g + (L-x_)*m1_hat*g + (L-x_)*m_hat*diff(H(t_,omega_N/N),t_,2) + M0*diff(H(t_,omega_N/N),t_,2)
    return lambdify((x_, t_), P_(x_,t_), 'numpy')(x,t)

In [9]:
# sway motion of the barge
def S(t, del_omega):
    return (eta(omega_p, omega_N/N)*np.sin(e1*omega_p*t+uniform(0, 2*np.pi))+
            np.sum(e2*eta(i*del_omega,del_omega)*np.sin(e1*i*del_omega*t+uniform(0, 2*np.pi)) for i in range(int(omega_p/del_omega),N+1)))

In [10]:
def q_hat(t, x, del_omega, ddv):
    z = -x
    
    def du(z,t,del_omega):
        return sum( (i*del_omega)**2*eta(i*del_omega, del_omega)*np.exp((i*del_omega)**2/g*(z+d)*np.sin(i*del_omega*t+uniform(0, 2*np.pi))/np.sinh((i*del_omega)**2/g*d))
                   for i in range(1,N+1))
    
    return 1.1*0.25*np.pi*CM*rho*D**2*du(z,t,omega_N/N)-0.25*np.pi*CA*rho*D**2*ddv

In [11]:
q_hat(0.1, 10, omega_N/N, .015)

655868.5332760524

### Finite difference solution

In [12]:
# EI*ddddv_ddddx-dP_dx*dv_dx-P*ddv_ddx+m_hat*ddv=q_hat(x,t)

In [13]:
T = 10  # Maximum time bound
del_t = 0.1
m = 50  # No. of intervals in space
n = int(T/del_t)  # No. of intervals in time
h = L / m  # Step size in space
k = T / n  # Step size in time


In [14]:
# Initialization of solution
v = np.zeros((m + 1, n + 1))

# Initial condition
ic1 = lambda x: S(x, omega_N/N) 

# Space discretization
for j in range(1, n + 2):
    v[0, j - 1] = ic1((j - 1) * h)

In [15]:
# Boundary condition
b1 = lambda t: 0  # U.B.C
b2 = lambda t: 0  # D.B.C

In [16]:
# Time discretization
for i in range(1, m + 2):
    v[i - 1, 0] = b1((i - 1) * k)
    v[i - 1, n] = b2((i - 1) * k)

In [17]:
v.shape

(51, 101)

In [18]:
# Time discretization
for i in range(1, n + 1):
    v[0, i] = b1((i - 1) * k)

In [19]:
y2=Symbol('y2')

In [20]:
def ddddv_ddddx(y0,y1,y2,y3,y4,h):
    return (y0-4*y1+6*y2-4*y3+y4)/h**4

In [21]:
def ddv_ddx(y1,y2,y3, h):
    return (y1-2*y2+y3)/h**2

In [22]:
def dv_dx(y1,y2,h):
    return (y2-y1)/(2*h)

In [23]:
def dP_dx(x1,x2,h,t):
    return (P(x2,t)-P(x1,t))/(2*h)

In [24]:
def gov_eq(y0, y1, y2, y3, y4,t0,t2, h, k, x,t):
    return (EI*ddddv_ddddx(y0,y1,y2,y3,y4,h)-dP_dx(x,x+h,h,t)*dv_dx(y1,y2,h)
            -P(x,t)*ddv_ddx(y1,y2,y3, h)+m_hat*ddv_ddx(t0,y2,t2, k)-q_hat(x,t,omega_N/N,ddv_ddx(t0,y2,t2, k)))

In [ ]:
startTime1 = datetime.now()
max_v = -float('inf')
res = np.zeros(v.shape)
for i in range(1, m-1):
    for j in range(1, n):
        res[i][j] = solve(gov_eq(v[i - 2, j], v[i - 1, j], y2, v[i +1, j], v[i + 2, j], v[i , j-1], v[i , j+1], h, k, i*h,j*k), y2)[0]
        max_v = max(max_v,res[i][j])  
print(datetime.now() - startTime1)        

In [ ]:
max_v

In [ ]:
if max_v/L<1:
    print("Stable")
else:
    print("Unbounded pipe response")

In [ ]:
# Visualization
x_ = np.linspace(0, L, m + 1 )
t_ = np.linspace(0, T, n + 1)
X, T = np.meshgrid(x_, t_)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, T, res.T, cmap='viridis')
ax.set_xlabel('Space X')
ax.set_ylabel('Time T')
ax.set_zlabel('V')
plt.title('Python Program for OTEC pipe')
plt.show()

Other references:

https://medium.com/@ghartirem072/scientific-computing-with-python-application-of-finite-difference-method-70fd9edff112

https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter23.03-Finite-Difference-Method.html